In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Preprocessing

In [2]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install nltk
!python -m nltk.downloader punkt

     |████████████████████████████████| 12.8 MB 8.4 MB/s eta 0:00:01    |████████████████▍               | 6.5 MB 8.4 MB/s eta 0:00:01     |████████████████████▏           | 8.1 MB 8.4 MB/s eta 0:00:01     |██████████████████████▋         | 9.1 MB 8.4 MB/s eta 0:00:01     |████████████████████████▌       | 9.8 MB 8.4 MB/s eta 0:00:01     |████████████████████████████▌   | 11.4 MB 8.4 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
/home/akshay/.conda/envs/research-work-DAG-DNN/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /home/akshay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import spacy
import string

In [4]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import random
 
#the stemmer requires a language parameter
snow_stemmer = SnowballStemmer(language='english')
word_tokenize("I am akshay")

['I', 'am', 'akshay']

In [5]:
STRT = "<\start>"
END = "<\end>"

In [6]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'my', 'both', 'everything', 'there', 'ca', 'elsewhere', 'show', 'whoever', 'therein', 'get', 'whence', 'others', 'hereafter', 'any', 'whenever', 'former', 'back', 'due', 'except', 'whose', 'quite', 'unless', 'into', 'those', 'anyway', 'other', 'indeed', 'name', 'say', "'d", 'side', 'make', 'under', 'third', 'at', 'using', 'how', '’s', 'why', 'next', 'before', 'made', 'i', 'wherein', 'than', 'noone', 'could', 'below', 'twelve', 'please', 'ten', 'much', 'three', 'amongst', 'against', 'often', 'sometime', 'themselves', 'give', 'were', 'most', 'twenty', 'neither', 'well', 'yourselves', 'out', 'along', 'they', 'also', 'becoming', 'wherever', 'amount', 'whereupon', 'though', 'when', 'your', 'herself', 'us', 'otherwise', 'him', 'up', 'none', 'ours', 'doing', 'on', 'one', 'all', 'so', 'without', 'it', 'yourself', 'if', 'just', '’ll', 'take', 'would', 'of', 'part', 'while', 'thence', 'already', 'them', 'never', 'a', '‘re', 'seemed', 'various', 'have', 'keep', "n't", 'n’t', 'call', 'these', 'up

In [7]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [8]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [9]:
glove_twitter_200_wv = api.load('glove-twitter-200')
gnews_300_wv = api.load('word2vec-google-news-300')

In [ ]:
def mean_wordvector(wv,keys):
    cnt = 0
    mean_wv = None
    for k in keys:
        cv = wv[k]
        if mean_wv is None:
            mean_wv = cv
        else:
            mean_wv = (mean_wv * (cnt-1) + cv)/cnt
        cnt+=1
    
    return mean_wv

In [ ]:
unknown_rep_glove_twitter_200 = mean_wordvector(glove_twitter_200_wv,glove_twitter_200_wv.key_to_index.keys())

In [ ]:
unknown_rep_gnews_300 = mean_wordvector(gnews_300_wv,gnews_300_wv.key_to_index.keys())

In [ ]:
glove_twitter_200_wv.most_similar("cricket"),gnews_300_wv.most_similar("cricket")

([('england', 0.6504806280136108),
  ('rugby', 0.6489739418029785),
  ('lanka', 0.6265019774436951),
  ('ipl', 0.6233053803443909),
  ('sachin', 0.6209328770637512),
  ('odi', 0.6159676313400269),
  ('football', 0.6020534038543701),
  ('batting', 0.601995587348938),
  ('pakistan', 0.597729504108429),
  ('footy', 0.5967651605606079)],
 [('cricketing', 0.8372225761413574),
  ('cricketers', 0.8165745735168457),
  ('Test_cricket', 0.8094819784164429),
  ('Twenty##_cricket', 0.8068488240242004),
  ('Twenty##', 0.7624265551567078),
  ('Cricket', 0.7541396617889404),
  ('cricketer', 0.7372578382492065),
  ('twenty##', 0.7316356897354126),
  ('T##_cricket', 0.7304614186286926),
  ('West_Indies_cricket', 0.698798656463623)])

In [ ]:
def spacy_lemmatizer(sentence,is_remove_stopword=False,is_remove_punct=False,verbose=False):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    if(verbose):
        print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if (not is_remove_stopword or word not in stop_words) and (not is_remove_punct or word not in punctuations) ]

    # return preprocessed list of tokens
    return mytokens

In [ ]:
def nltk_stemmer(stemmer_obj,sentence,is_remove_stopword=False,is_remove_punct=False):
    mytokens = word_tokenize(sentence)
    # print(mytokens)
    # Removing stop words and punctuations
    mytokens = [ stemmer_obj.stem(word) for word in mytokens if (not is_remove_stopword or word not in stop_words) and (not is_remove_punct or word not in punctuations) ]

    # return preprocessed list of tokens
    return mytokens

#### Different variations during tokenization

##### Lemmatization

In [ ]:
# With removing stopwords
spacy_lemmatizer("This is Bangalore. I am living here!",True,verbose=True)

['this', 'be', 'bangalore', '.', 'i', 'be', 'live', 'here', '!']


['bangalore', '.', 'live', '!']

In [ ]:
# With removing punctuations
spacy_lemmatizer("This is Bangalore. I am living here!",False,True,verbose=True)

['this', 'be', 'bangalore', '.', 'i', 'be', 'live', 'here', '!']


['this', 'be', 'bangalore', 'i', 'be', 'live', 'here']

In [ ]:
# With removing stopwords and punctuations
spacy_lemmatizer("This is Bangalore. I am living here!",True,True,verbose=True)

['this', 'be', 'bangalore', '.', 'i', 'be', 'live', 'here', '!']


['bangalore', 'live']

In [ ]:
# Without removing punctuation and stopwords
spacy_lemmatizer("This is Bangalore. I am living here!",verbose=True)

['this', 'be', 'bangalore', '.', 'i', 'be', 'live', 'here', '!']


['this', 'be', 'bangalore', '.', 'i', 'be', 'live', 'here', '!']

We will not be removing the stopwords and punctuations since it will be beneficial for seq2seq architecture. The other lemmatization variations with stopwords and punctuations are more useful for simpler models.

##### Stemming

In [ ]:
nltk_stemmer(snow_stemmer,"This is Bengaluru. I am living here!")

['this', 'is', 'bengaluru', '.', 'i', 'am', 'live', 'here', '!']

Stemming is sufficient especially when corpus is large since it is very fast.

#### Sentence Vectorization
This is applied over the tokens.
This can be done either by certain dedicated algorithms like Bag of words,TF-IDF or by taking the average word representation in sentence using word representations like Word2Vec,Glove etc..\\
We will be using word representation since we will be using seq2seq

In [ ]:
# Skips UNKNOWN words while generating sentence representation
def average_word_rep_sentence_vectorizer(sent,word_vector_obj_list):
    res_list = []
    for word_vector_obj in word_vector_obj_list:
        vector_size = word_vector_obj.vector_size
        wv_res = np.zeros(vector_size)
        # print(wv_res)
        ctr = 1
        for w in sent:
            if w in word_vector_obj:
                ctr += 1
                wv_res += word_vector_obj[w]
        wv_res = wv_res/ctr
    
    return wv_res

def 

In [ ]:
tst1 = np.random.rand(3,5)
tst2 = np.random.rand(3,4)
print(tst1,tst2)
tst = np.stack([tst1,tst2])
print(tst,tst.shape)

[0.32984137 0.38722697 0.17904217 0.77158292 0.37752822] [0.22577246 0.71257047 0.70641463 0.02552281]


ValueError: all input arrays must have the same shape

### Dataset read

In [ ]:
import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/CNN-DailyMail News Dataset/train.csv
./data/CNN-DailyMail News Dataset/validation.csv
./data/CNN-DailyMail News Dataset/test.csv


In [ ]:
train_data = pd.read_csv('./data/CNN-DailyMail News Dataset/train.csv')
test_data = pd.read_csv('./data/CNN-DailyMail News Dataset/test.csv')
valid_data = pd.read_csv('./data/CNN-DailyMail News Dataset/validation.csv')

In [ ]:
train_data.sample(5)

,id,article,highlights
237876,bfe46158407b71846f408b141d3276d68bc86665,Football was hit by a fresh racism storm after...,Feyenoord's Europa League match with Roma was ...
222245,abb2b894628c02b20cddf53c36957532593a9407,"By . Associated Press . and David Martosko, U....",Obama appointed three people to the National L...
9186,1a082af1a725f85b460b141a76e8d85c9760cf9f,A stunning catalog of torture and the widespre...,U.N. report warns North Korean leader Kim Jong...
241992,c52dca6cb45e97628b6f309f70990fca5acabd33,A Florida woman who tried to shoot a snake wit...,"April DeMarco, 30, was watching a football pra..."
154607,53ca5d75870b33ede68cb51340e40fe2ba904733,"(CNN) -- Thousands of Ontario, Canada, residen...","NEW: Power restored to 100,000 customers in To..."


In [ ]:
def get_extra_token_rep(wv,num_samples):
    random_indices=np.array(random.sample(range(0,len(wv)), num_samples))
    klist = [wv.index_to_key[k] for k in random_indices]
    return mean_wordvector(wv,klist)

In [ ]:
random.seed(10)
glove_twitter_200_wv[STRT] = get_extra_token_rep(glove_twitter_200_wv,10)
glove_twitter_200_wv[END] = get_extra_token_rep(glove_twitter_200_wv,10)
gnews_300_wv[STRT] = get_extra_token_rep(gnews_300_wv,10)
gnews_300_wv[END] = get_extra_token_rep(gnews_300_wv,10)

id                     0001d1afc246a7964130f43ae940af6bc6c57f01
article       By . Associated Press . PUBLISHED: . 14:11 EST...
highlights    Bishop John Folda, of North Dakota, is taking ...
Name: 0, dtype: object

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class TextSummarizationDataset(Dataset):
    def __init__(self, pandas_frame,src_transform=None,target_transform=None,src_key="article",target_key="highlights"):
        self.pandas_frame = pandas_frame
        self.src_key = src_key
        self.target_key = target_key
        self.src_transform = src_transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.pandas_frame)

    def __getitem__(self, idx):
        self.source_vec,self.target_vec = self.pandas_frame.iloc[idx][self.src_key],self.pandas_frame.iloc[idx][self.target_key]
        
        if self.src_transform:
            self.source_vec = self.src_transform(self.source_vec)
        if self.target_transform:
            self.target_vec = self.target_transform(self.target_vec)
        
        return self.source_vec, self.target_vec